# **Modeling**
public leaderboard score: 0.705   
kaggle notebook v3   
Rank 1636 , Jan 2, 2024  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

import re
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin 

#metrics
from sklearn.metrics import mean_squared_error
#model selection
from sklearn.model_selection import cross_val_score

#load preprocessed dataset:
import joblib
#models
from sklearn.linear_model import LinearRegression, ElasticNet 
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')
test_logs = pd.read_csv(input_folder + "test_logs.csv",delimiter = ",",header = 0)

In [3]:
# Feature Engineering for transformer for cursor position
class CursorPositionTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        # setup the feature names
        # self.feature_names = ['cp_sum_backstep', 'cp_n_backstep', 'cp_sum_forwardstep','cp_n_forwardstep',
        #              'cp_change_stat', 'cp_skew_backstep', 'cp_skew_forwardstep']  

        self.feature_names = ['cp_sum_backstep', 'cp_n_backstep', 'cp_sum_forwardstep','cp_n_forwardstep']

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        z = X.groupby('id')['cursor_position'].aggregate([self.cp_sum_backstep,self.cp_n_backstep, 
                     self.cp_sum_forwardstep, self.cp_n_forwardstep])
        # make a copy of participant ids:
        self.index_ids = z.index.values
        return z.values

    def cp_sum_backstep(self,x):
        n1 = np.diff(np.log(x+1))
        return np.sum(n1[n1 < 0])
    
    def cp_skew_backstep(self,x):
        n1 = np.diff(np.log(x+1))
        return st.skew(n1[n1 < 0])
    
    def cp_n_backstep(self,x):
        n1 = np.diff(np.log(x+1))
        return np.log((n1<0).sum()+1)
    
    def cp_sum_forwardstep(self,x):
        n1 = np.diff(np.log(x+1))
        return np.sum(n1[n1 > 0])
    
    def cp_skew_forwardstep(self,x):
        n1 = np.diff(np.log(x+1))
        return st.skew(n1[n1 > 0])
    
    def cp_n_forwardstep(self,x):
        n1 = np.diff(np.log(x+1))
        return np.log((n1>0).sum()+1)
    
    def cp_change_stat(self,x):
        n1 = np.diff(np.log(x+1))
        return np.std(n1, ddof = 1)
    

In [4]:
# eda wordcount transformer:

# word_count feature engineering
# Based on the graph above, we can count the number of zero changes and get the mean:
# wc_zero_change will return the count of all non-zero steps taken by the person

class WordCountTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y = None):
        return self

    def wc_non_zero_change(self, x):
        n1 = np.diff(np.log(x+1))
        n2 = np.count_nonzero(n1)
        return n2
    def wc_change_stat(self, x):
        n1 = np.diff(np.log(x+1))
        last_cutoff = n1.shape[0]-200
        n2 = np.std(n1, ddof = 1)
        return n2
        
    def transform(self, X):
        output =  X.groupby(['id'])['word_count'].aggregate([self.wc_non_zero_change,lambda x: np.log(len(x)), 
                          lambda x: np.log(np.max(x)+1)])
        output.columns = ["wc_changing_nsteps", "wc_step_count", "wc_max"]
        self.feature_names = output.columns.values
        self.index_ids = output.index.values
        return output.values
        

In [5]:
# eda textchange transformer:
class TextChangeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y = None):
        return self

    def hasChar(self, x,character:str):
        out = 0 
        for strings in x:
            if character in strings:
                out = 1
                break
        return out
        
    def transform(self, X):
        output = X.groupby(['id'])['text_change'].aggregate([
            ("tc_1", lambda x: self.hasChar(x,character = "?")), 
            ("tc_2", lambda x: self.hasChar(x,character = "=>")), 
            ("tc_3", lambda x: self.hasChar(x,character = "(")), 
            ("tc_4", lambda x: self.hasChar(x,character = "\"")), 
            ("tc_5", lambda x: self.hasChar(x,character = "-"))]) 
        self.feature_names = output.columns.values
        self.index_ids = output.index.values
        return output.values
        

            

In [6]:
# Feature Engineering Up Event Variable Transformer:
class UpEventTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y = None):
        return self


    
    def find_clicked(self, x, st:str):
        has_string = 0
        for event in x:
            if(event == st):
                has_string = 1
                break
        return has_string

    def transform(self, X):
        
        output = X.groupby(['id'])['up_event'].aggregate([('ue_1',lambda x: self.find_clicked(x,"|")),
                                                          ('ue_2', lambda x: self.find_clicked(x,"Shift")),
                                                          ('ue_3', lambda x: self.find_clicked(x,"Tab")),
                                                          ])
        self.feature_names = output.columns.values
        self.index_ids = output.index.values
        return output.values



In [7]:
# Eda action_time variable ransformer: (AT)

class ActionTimeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, scores):
        self.scores = scores
        self.score_values = np.arange(start = 0.5, stop = 6.5, step = 0.5)

    def fit(self, X, y = None):
        #Get the action time proportion or distribution per score:
        at_init = X.groupby('id')['action_time'].aggregate([
            ('one', lambda x: self.above_log_count(x, from_zero = 1)),
            ('two', lambda x: self.above_log_count(x, from_zero = 2)),
            ('three', lambda x: self.above_log_count(x, from_zero = 3)),
            ('four', lambda x: self.above_log_count(x, from_zero = 4)),
            ('five', lambda x: self.above_log_count(x, from_zero = 5)),
        ])
        
        at_init2 = pd.merge(at_init, self.scores, left_index = True, right_index = True)
        at2 = at_init2.groupby(by = 'score').sum()
        self.at_proportion= at2.apply(lambda x: x/(np.sum(at2, axis = 1)))
        return self
        
    def above_log_count(self, x, from_zero = 1):
        z = np.diff(np.log(x+1))
        z = np.abs(z)
        if from_zero < 5:
            count= len(list(filter(lambda q: (q>from_zero -1) and (q < from_zero), z)))
        else:
            count= len(list(filter(lambda q: q>=from_zero, z )))
        return count 
        
    def above_log_ratio(self, x, from_zero = 1):
        z = np.diff(np.log(x+1))
        z = np.abs(z)
        if from_zero < 3:
            count= len(list(filter(lambda q: (q>from_zero -1) and (q < from_zero), z)))
        else:
            count= len(list(filter(lambda q: q>=from_zero, z )))
        return np.log((count+1)/len(z)) 

        
    # Use chi-square to select the score of the given participant id   
    def compute_score_by_chisquare(self, fo:pd.Series, distribution):
        fo =fo + 1 # to remove errors for those with zero values
        total = np.sum(fo)
        # print(total)
        expected_arrays = distribution * total
        # print(expected_arrays)
        chi_stat = []
        for j in range(expected_arrays.shape[0]):
            results = st.chisquare(f_obs = fo, f_exp = expected_arrays.iloc[j])
            chi_stat.append(results[1])
    
        chi_stat = np.array(chi_stat)
        # get the maximum p-value (-1) or second to the max (-2), etc
        score_idx_1 = np.where(chi_stat == np.partition(chi_stat,-1)[-1])[0][0]
        score_idx_2 = np.where(chi_stat == np.partition(chi_stat,-2)[-2])[0][0]
        score_idx_3 = np.where(chi_stat == np.partition(chi_stat,-3)[-3])[0][0]
        score_idx_4 = np.where(chi_stat == np.partition(chi_stat,-4)[-4])[0][0]
        score_list = [
            self.score_values[score_idx_1],
            self.score_values[score_idx_3],
            self.score_values[score_idx_3],
            self.score_values[score_idx_4]]
        
        return np.mean(score_list)
        
    def transform(self, X):
        transform_1 = X.groupby("id")['action_time'].aggregate([
        ('at_1', lambda x: self.above_log_ratio(x, from_zero = 1)),
        ('at_2', lambda x: self.above_log_ratio(x, from_zero = 2)),
        ('at_3', lambda x: self.above_log_ratio(x, from_zero = 3))
        ])
        
        at_init = X.groupby('id')['action_time'].aggregate([
            ('one', lambda x: self.above_log_count(x, from_zero = 1)),
            ('two', lambda x: self.above_log_count(x, from_zero = 2)),
            ('three', lambda x: self.above_log_count(x, from_zero = 3)),
            ('four', lambda x: self.above_log_count(x, from_zero = 4)),
            ('five', lambda x: self.above_log_count(x, from_zero = 5)),
        ])
        transform_2 = at_init.apply(
            lambda x: self.compute_score_by_chisquare(x, distribution = self.at_proportion),axis = 1)
        transform_2.name = "at_chisq"
        output = pd.merge(transform_1, transform_2, left_index = True, right_index = True)
        self.feature_names = output.columns.values
        self.index_ids = output.index.values
        return output.values 

        
        


In [8]:
# Transformer for Activity, act:
class ActivityTransformer(BaseEstimator, TransformerMixin):
    oneHot: OneHotEncoder
    scores: pd.Series
    act_dist: pd.DataFrame
    feature_names: np.array
    initial_features: np.array
    
    def __init__(self, scores:pd.Series):
        self.oneHot = OneHotEncoder(handle_unknown = 'ignore', categories = 'auto', sparse_output = False)
        self.scores = scores
        self.score_values = np.arange(start = 0.5, stop = 6.5, step = 0.5)
        self.initial_features = np.array(['ac_Input', 'ac_Move', 'ac_NonPro', 'ac_Paste', 'ac_RemCut', 'ac_Replace'])
        
    def fit(self,X, y=None):
        #Transform X labels first:
        #Transform all with move into a Move:
        X.activity = X.activity.apply(lambda x: "Move" if ("Move" in x) else x)
        #Encode then get the distribution
        self.oneHot.fit(X)
        a1 = self.oneHot.fit_transform(X.activity.values.reshape(-1,1))
        a2 = pd.DataFrame(data=a1, columns=self.initial_features)
        a2['id'] = X.id.copy()
        
        act = a2.groupby(by = "id").sum()
        act = act + 1 # to avoid expected value of zero
        
        # Get the distribution for each kind of score
        # act distribution:
        act_dist = pd.merge(act, scores, left_index = True, right_index = True)
        act_dist = act_dist.groupby('score').sum()
        
        row_total = np.sum(act_dist, axis = 1)
        self.act_dist = act_dist.apply(lambda x: x / row_total)
            
        return self


    def compute_score_by_chisquare(self, fo:pd.Series, distribution):
        fo = fo+1
        total = np.sum(fo)
        # print(total)
        # add 1 to avoid expected value of zero.
        expected_arrays = distribution * total 
        # print(expected_arrays)
        chi_stat = []
        for j in range(expected_arrays.shape[0]):
            results = st.chisquare(f_obs = fo, f_exp = expected_arrays.iloc[j])
            chi_stat.append(results[1])
    
        chi_stat = np.array(chi_stat)
        # get the maximum p-value (-1) 
        score_idx_1 = np.where(chi_stat == np.partition(chi_stat,-1)[-1])[0][0]
        
        return self.score_values[score_idx_1]


    def transform(self, X):
        #Transform X labels first:
        #Transform all with move into a Move:
        X.activity = X.activity.apply(lambda x: "Move" if ("Move" in x) else x)
        
        pre_output = self.oneHot.transform(X['activity'].values.reshape(-1,1))
        a2 = pd.DataFrame(data = pre_output, columns = self.initial_features)
        a2['id'] = X.id 
        act = a2.groupby('id').sum()
        output = act.apply(lambda x: self.compute_score_by_chisquare(x, self.act_dist), axis = 1)
        output.name = "act_chisq"
        self.feature_names = output.name
        return output.values.reshape(-1,1)
        


In [9]:
# Pipeline to combine summary:
cp_pipe = Pipeline([('cp_tx', CursorPositionTransformer())])
wc_pipe = Pipeline([('wc_tx', WordCountTransformer())])
tc_pipe = Pipeline([('tc_tx', TextChangeTransformer())])
ue_pipe = Pipeline([('ue_tx', UpEventTransformer())])
at_pipe = Pipeline([('at_tx', ActionTimeTransformer(scores = scores))])
act_pipe = Pipeline([("act_tx", ActivityTransformer(scores))])

#join the pipes:
main_pipe = FeatureUnion(transformer_list = [
    ('cp_pipe', cp_pipe),
    ('wc_pipe', wc_pipe),
    ('tc_pipe', tc_pipe),
    ('ue_pipe', ue_pipe),
    ('at_pipe', at_pipe),
    ('act_pipe', act_pipe)])

final_pipe = Pipeline([('main_pipe', main_pipe),
                        ('Poly', PolynomialFeatures(degree = 2, include_bias = False)),
                        ('Scaler', StandardScaler())
                         ])


In [10]:
X = final_pipe.fit_transform(train_logs)

In [11]:
train_ids = main_pipe.named_transformers['cp_pipe'].named_steps['cp_tx'].index_ids

# **Modeling Portion**

In [12]:
#dum main pipe
#joblib.dump(X, "transformed_train.pkl")
#joblib.dump(train_ids, "train_ids.pkl")

In [13]:
Y = scores.values
Y.shape

(2471,)

In [18]:
model = BaggingRegressor(estimator = SVR(C=10000, epsilon = 0.0001), max_samples = 400, 
                         n_estimators = 100, random_state = 11)
model.fit(X,Y)
#remove below:
rmse = np.sqrt(mean_squared_error(model.predict(X), Y))
print("RMSE: {}".format(rmse))

RMSE: 0.6059289685412605


In [15]:
# Prediction:

X_test = final_pipe.transform(test_logs)
test_ids = test_logs.id.unique()

In [16]:
prediction = model.predict(X_test)
submission = pd.DataFrame({'id':test_ids, 'score':prediction})

In [17]:
submission.to_csv("submission.csv", index = False)